# Final Project
### Import precomp libs

In [1]:
import spacy
nlp = spacy.load('en') # this should take some time like 10s to load
%load_ext line_profiler

In [2]:
test_loc = 'test_dump.bin'
train_loc = 'train_dump.bin'

In [3]:
from spacy.tokens.doc import Doc
import pickle 
import numpy as np
import scipy as sp
import pandas as pd
from collections import Counter
from sklearn.feature_extraction import DictVectorizer
from scipy.sparse import coo_matrix, hstack
from datetime import datetime

In [59]:
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression, SGDClassifier, RidgeClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.multiclass import OneVsRestClassifier

In [33]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')
print train.shape, test.shape
target = train['is_duplicate']

(404290, 6) (2345796, 3)


In [5]:
docs = []
i = 0
print datetime.now().strftime('%Y-%m-%d %H:%M:%S')
with open(train_loc, 'rb') as file_:
    for byte_string in Doc.read_bytes(file_):
        if i%10000 == 0: print i, datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        docs.append(Doc(nlp.vocab).from_bytes(byte_string))
        i += 1
#         if i == 10: break
print datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print(type(docs[0]))

2017-05-14 16:01:18
0 2017-05-14 16:01:18
10000 2017-05-14 16:01:21
20000 2017-05-14 16:01:22
30000 2017-05-14 16:01:23
40000 2017-05-14 16:01:24
50000 2017-05-14 16:01:25
60000 2017-05-14 16:01:26
70000 2017-05-14 16:01:27
80000 2017-05-14 16:01:28
90000 2017-05-14 16:01:29
100000 2017-05-14 16:01:30
110000 2017-05-14 16:01:31
120000 2017-05-14 16:01:32
130000 2017-05-14 16:01:33
140000 2017-05-14 16:01:34
150000 2017-05-14 16:01:35
160000 2017-05-14 16:01:35
170000 2017-05-14 16:01:36
180000 2017-05-14 16:01:37
190000 2017-05-14 16:01:38
200000 2017-05-14 16:01:39
210000 2017-05-14 16:01:40
220000 2017-05-14 16:01:41
230000 2017-05-14 16:01:42
240000 2017-05-14 16:01:43
250000 2017-05-14 16:01:44
260000 2017-05-14 16:01:45
270000 2017-05-14 16:01:45
280000 2017-05-14 16:01:46
290000 2017-05-14 16:01:47
300000 2017-05-14 16:01:48
310000 2017-05-14 16:01:49
320000 2017-05-14 16:01:50
330000 2017-05-14 16:01:51
340000 2017-05-14 16:01:52
350000 2017-05-14 16:01:53
360000 2017-05-14 16:0

In [28]:
def wordPairs(docs):
    fvs = []
    for i in range(0,len(docs),2):
        if i % 100000 == 0: print i
        fv = {}
        d1 = docs[i]
        d2 = docs[i+1]
        
        s1 = set([word.lemma_ for word in d1 if not word.is_stop and word.pos is not 95])
        s2 = set([word.lemma_  for word in d2 if not word.is_stop and word.pos is not 95])
        diff = s1 ^ s2
        same =  s1 & s2

        same = Counter([w for w in same])
        fv.update(same)

        diff = Counter([w + '_d' for w in diff])
        fv.update(diff)

        fvs.append(fv)
        
    return fvs

In [29]:
fvs = wordPairs(docs)

0
100000
200000
300000
400000
500000
600000
700000
800000


In [88]:
from sklearn.feature_extraction.text import TfidfTransformer
v = DictVectorizer()
t = TfidfTransformer(smooth_idf=False)

In [89]:
X = v.fit_transform(fvs)
# X = t.fit_transform(counts)

In [90]:
X.shape

(404290, 112503)

#### Validation

In [94]:
# Training classifiers

clf1 = MultinomialNB()
clf2 = SGDClassifier(loss='log')
clf3 = LogisticRegression()
eclf = VotingClassifier(estimators=[('mn', clf1), ('sgd', clf2), ('lr', clf3)], voting='soft', weights=[1,1,4])


scores = cross_val_score(eclf, X, target, cv=3, scoring='neg_log_loss')
print "Log Loss: %0.5f" % scores.mean()

Log Loss: -0.41635


In [93]:
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
clfs = [MultinomialNB(alpha=1), SGDClassifier(loss='log'), LogisticRegression(), RandomForestClassifier(n_estimators=10, criterion='gini', max_depth=50, min_samples_split=30, min_samples_leaf=8, min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, min_impurity_split=1e-07, bootstrap=True, oob_score=False, n_jobs=-1, warm_start=True, class_weight=None)]

for clf in clfs:
    print clf
    scores = cross_val_score(clf, X, target, cv=3, scoring='neg_log_loss')
    print "Log Loss: %0.5f" % scores.mean()

MultinomialNB(alpha=1, class_prior=None, fit_prior=True)
Log Loss: -0.51773
SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)
Log Loss: -0.46116
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
Log Loss: -0.41490
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=50, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=8,
            min_samples_split=30, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=-1, oob_score=False, rand

### Testing

In [ ]:
test_docs = []
i = 0
with open(test_loc, 'rb') as file_:
    for byte_string in Doc.read_bytes(file_):
        test_docs.append(Doc(nlp.vocab).from_bytes(byte_string))
        i += 1

In [98]:
f = open('submission', 'w')
f.write('test_id,is_duplicate\n')

In [ ]:
fv_test = wordPairs(test_docs)

In [ ]:
fv_test = v.transform(fv_test)

In [ ]:
clf = LogisticRegression().fit(X, target)
prob = clf.predict_proba(fv_test)
for i, p in enumerate(prob):
    f.write(str(i) + ',' + str(p[0]) + '\n')